# Analizamos y transformamos el dataset "titanic_v2023"

Vamos a analizar el dataset para ver que transformaciones hay que aplicar para que este listo para ser usado.

Descargar el dataset por medio de un link publico.

In [1]:
import sys

# Instalar gdown si no está instalado
if 'gdown' not in sys.modules:
    !pip install gdown -q

In [2]:
# ID del archivo público de Google Drive
file_id = '1cYsJdV4qlxSLuppfhlTAZuLZ_kl_m7sz'
filename = 'titanic_v2023.csv'

# Descargar el archivo
!gdown --id {file_id} -O {filename}

/home/felix/anaconda3/lib/python3.13/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1cYsJdV4qlxSLuppfhlTAZuLZ_kl_m7sz
To: /home/felix/Repositories/Python/data-science/notebooks/titanic_v2023.csv
100%|██████████████████████████████████████| 38.0k/38.0k [00:00<00:00, 2.39MB/s]


In [3]:
import os
import shutil

# Nombre de la carpeta
downloads_folder = 'downloads'

# Crear la carpeta 'downloads' si no existe
if not os.path.exists(downloads_folder):
    os.makedirs(downloads_folder)
    print(f"Carpeta '{downloads_folder}' creada exitosamente.")
else:
    print(f"La carpeta '{downloads_folder}' ya existe.")

# Nombre del archivo y su ruta original
source_path = os.path.join(os.getcwd(), filename)

# Ruta de destino del archivo
destination_path = os.path.join(os.getcwd(), downloads_folder, filename)

# Mover el archivo a la carpeta 'downloads'
if os.path.exists(source_path):
    shutil.move(source_path, destination_path)
    print(f"Archivo '{filename}' movido a '{downloads_folder}/' exitosamente.")
else:
    print(f"El archivo '{filename}' no se encontró en la ruta original.")


Carpeta 'downloads' creada exitosamente.
Archivo 'titanic_v2023.csv' movido a 'downloads/' exitosamente.


In [4]:
import pandas as pd

# Leer el archivo CSV
titanic2023 = pd.read_csv(os.path.join(downloads_folder, filename))
titanic2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 10 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   sex                                     889 non-null    object
 1   age                                     712 non-null    object
 2   siblings of the passenger	              889 non-null    int64 
 3   parents / children aboard the Titanic	  889 non-null    int64 
 4   fare                                    889 non-null    object
 5   Port of Embarkation	                    889 non-null    object
 6   class                                   889 non-null    object
 7   who                                     889 non-null    object
 8   alone                                   889 non-null    bool  
 9   survived                                889 non-null    object
dtypes: bool(1), int64(2), object(7)
memory usage: 63.5+ KB


In [5]:
titanic2023.head()

,sex,age,siblings of the passenger\t,parents / children aboard the Titanic\t,fare,Port of Embarkation\t,class,who,alone,survived
0,male,39,0,0,"7,18",S,First,man,True,no
1,male,20,0,0,"4,0125",C,Third,man,True,no
2,male,33,0,0,5,S,First,man,True,no
3,male,61,0,0,"6,2375",S,Third,man,True,no
4,male,"34,5",0,0,"6,4375",C,Third,man,True,no


# Análisis

Se puede observar que el dataset tiene datos erroneos, como edades intermedias y valores faltantes en la columna "age", tambien los nombres de algunas columnas son muy largos y contienen secuencias de escape("\t"), lo cual no tiene sentido como nombre de una columna.

## Columna edad

Esta columna tiene varios errores, tiene datos nulos y es de tipo objeto(supongo que un string) cuando deberia ser un numero entero.
Ademas la columna de edad tiene valores intermedios los cuales son incorrectos, ej: no tiene sentido que una persona tenga 45.5 años

## Columna fare

Esta columna es de tipo string pero deberia ser float.

## Columna Port of embarkation

Puerto de embarque en español, contiene varios problemas en el nombre; Primero que es muy largo, deberia simplemente llamarse "embarkation_port" y segundo que al final tiene un escape de tabulador "Port of embarkation\t", esto solo se puede ver al usar .head(), cuando se usa .info(), esto no es visible ya que se muestra el escape de tabulador en vez de "\t".

Esto que implica es que al tratar de usar la columna no es lo mismo escribir titanic2023_copia["Port of embarkation"] que titanic2023_copia["Port of embarkation\t"] (notemos el \t al final), este mismo problema lo tiene la columna de hermanos(siblings) y la de padres e hijos(parents / children).

# Datos duplicados y datos nulos

La columna "age" del dataset tiene 105 datos duplicados, los cuales vamos a  remover.

In [6]:
# Analizar si existen duplicados
cantidad_duplicados = titanic2023.duplicated().sum()
print("Cantidad de filas duplicadas: ", cantidad_duplicados)

# Creamos una copia del dataset para manipularlo
titanic2023_copia = titanic2023.copy()

# Remover duplicados
titanic2023_copia = titanic2023_copia.drop_duplicates()

cantidad_duplicados = titanic2023_copia.duplicated().sum()
print("Filas duplicadas despues de removerlas: ", cantidad_duplicados)

# Datos nulos o faltantes
print("Cuantos datos faltantes hay")
cantidad_faltantes = titanic2023_copia.isnull().sum()
cantidad_faltantes

Cantidad de filas duplicadas:  105
Filas duplicadas despues de removerlas:  0
Cuantos datos faltantes hay


sex                                          0
age                                        110
siblings of the passenger\t                  0
parents / children aboard the Titanic\t      0
fare                                         0
Port of Embarkation\t                        0
class                                        0
who                                          0
alone                                        0
survived                                     0
dtype: int64

# Corregir datos erroneos

En esta seccion corregiremos varios errores del dataset.

Primero arreglamos los nombres de las columnas los cuales son muy largos y no son apropiados para un dataset.

Segundo transformamos los valores de tipo objeto a tipo entero o punto flotante(float).

In [7]:
import numpy as np

# Arreglamos el nombre de las columnas para que se vea mejor y sea
# mas facil de acceder
titanic2023_copia = titanic2023_copia.rename(
    columns={
        "Port of Embarkation\t": "embarkation_port",
        "siblings of the passenger\t": "passenger_siblings",
        "parents / children aboard the Titanic\t": "parents_children_aboard"
    }
)

# En ingles se usa un punto en vez de una coma para identificar valores decimales
# por lo tanto tenemos que cambiar las comas por puntos
titanic2023_copia["age"] = titanic2023_copia["age"].str.replace(",", ".", regex=False).astype(float)

# Ahora redondeamos hacia abajo y convertimos a entero.
# Int64 puede tener valores nulos a diferencia de int64
titanic2023_copia["age"] = np.floor(titanic2023_copia["age"]).astype("Int64")

# Transformamos la columna fare(tarifa) a decimal
titanic2023_copia["fare"] = titanic2023_copia["fare"].str.replace(",", ".", regex=False).astype(float)

titanic2023_copia.head()

,sex,age,passenger_siblings,parents_children_aboard,fare,embarkation_port,class,who,alone,survived
0,male,39,0,0,7.1800,S,First,man,True,no
1,male,20,0,0,4.0125,C,Third,man,True,no
2,male,33,0,0,5.0000,S,First,man,True,no
3,male,61,0,0,6.2375,S,Third,man,True,no
4,male,34,0,0,6.4375,C,Third,man,True,no


# Aplicar One-Hot Encoding

Usamos One-Hot Encoding para la columna "class" y otras las cuales podrían ser útiles

In [8]:
# Aplicamos one-hot encoding
# Las variables categoricas son "sex", "who", "class", "survived"
titanic2023_encoded = pd.get_dummies(
    titanic2023_copia,
    columns=[
        "sex",
        "who",
        "class",
        "survived",
        "embarkation_port"
    ],
    dtype=int
)

# Mostramos la informacion cambiada para verificar que todo este bien
titanic2023_encoded.head()

# Dataset con las correciones aplicadas y encodeado para la imputacion
titanic2023_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 784 entries, 0 to 888
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      674 non-null    Int64  
 1   passenger_siblings       784 non-null    int64  
 2   parents_children_aboard  784 non-null    int64  
 3   fare                     784 non-null    float64
 4   alone                    784 non-null    bool   
 5   sex_female               784 non-null    int64  
 6   sex_male                 784 non-null    int64  
 7   who_child                784 non-null    int64  
 8   who_man                  784 non-null    int64  
 9   who_woman                784 non-null    int64  
 10  class_First              784 non-null    int64  
 11  class_Second             784 non-null    int64  
 12  class_Third              784 non-null    int64  
 13  survived_no              784 non-null    int64  
 14  survived_yes             784 no

# Imputar datos faltantes

Rellenamos valores faltantes con KNNImputer.

Vemos que luego de esta operacion no hay mas datos faltantes, y la columna de edad ha sido convertida a tipo float, esta conversion no nos causa problemas por ahora.


In [9]:
from sklearn.impute import KNNImputer

# Creamos una copia del data frame
titanic2023_imputado = titanic2023_encoded.copy()

# Identificamos las columnas numericas que necesitan imputacion
imputar_cols = ["age"]

# Imputamos valores con KNN
imputer = KNNImputer(n_neighbors=5)
titanic2023_imputado[imputar_cols] = imputer.fit_transform(titanic2023_imputado[imputar_cols])

# Vemos si hay valores nulos ahora
cantidad_faltantes = titanic2023_imputado.isnull().sum()
print("Cantidad de datos faltantes")
print(cantidad_faltantes)

# No hay mas datos faltantes
titanic2023_imputado

Cantidad de datos faltantes
age                        0
passenger_siblings         0
parents_children_aboard    0
fare                       0
alone                      0
sex_female                 0
sex_male                   0
who_child                  0
who_man                    0
who_woman                  0
class_First                0
class_Second               0
class_Third                0
survived_no                0
survived_yes               0
embarkation_port_C         0
embarkation_port_Q         0
embarkation_port_S         0
dtype: int64


,age,passenger_siblings,parents_children_aboard,fare,alone,sex_female,sex_male,who_child,who_man,who_woman,class_First,class_Second,class_Third,survived_no,survived_yes,embarkation_port_C,embarkation_port_Q,embarkation_port_S
0,39.0,0,0,7.1800,True,0,1,0,1,0,1,0,0,1,0,0,0,1
1,20.0,0,0,4.0125,True,0,1,0,1,0,0,0,1,1,0,1,0,0
2,33.0,0,0,5.0000,True,0,1,0,1,0,1,0,0,1,0,0,0,1
3,61.0,0,0,6.2375,True,0,1,0,1,0,0,0,1,1,0,0,0,1
4,34.0,0,0,6.4375,True,0,1,0,1,0,0,0,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,24.0,3,2,263.0000,False,1,0,0,0,1,1,0,0,0,1,0,0,1
885,64.0,1,4,263.0000,False,0,1,0,1,0,1,0,0,1,0,0,0,1
886,35.0,0,0,512.3292,True,1,0,0,0,1,1,0,0,0,1,1,0,0
887,36.0,0,1,512.3292,False,0,1,0,1,0,1,0,0,0,1,1,0,0


# Reescalamos las columnas no categóricas

El reescalado de datos es un paso crucial para algoritmos de Machine Learning, evita el dominio de características con valores grandes, mejora la convergencia del algoritmo y facilita la regularización.

Las columnas no categóricas son "age", "fare"(tarifa), "passenger_siblings"(hermanos de pasajeros) y "parents_children_aboard"(si los padres e hijos han abordado).

In [10]:
from sklearn.preprocessing import StandardScaler

# Que columnas vamos a reescalar
columnas_a_reescalar = ["age", "fare", "passenger_siblings", "parents_children_aboard"]

# Creamos otra copia del DataFrame
titanic2023_reescalado = titanic2023_imputado.copy()

# Reescalar columnas
scaler = StandardScaler()
titanic2023_reescalado[columnas_a_reescalar] = scaler.fit_transform(titanic2023_reescalado[columnas_a_reescalar])

titanic2023_reescalado.head()

,age,passenger_siblings,parents_children_aboard,fare,alone,sex_female,sex_male,who_child,who_man,who_woman,class_First,class_Second,class_Third,survived_no,survived_yes,embarkation_port_C,embarkation_port_Q,embarkation_port_S
0,0.680653,-0.530599,-0.497157,-0.536112,True,0,1,0,1,0,1,0,0,1,0,0,0,1
1,-0.714424,-0.530599,-0.497157,-0.597004,True,0,1,0,1,0,0,0,1,1,0,1,0,0
2,0.240102,-0.530599,-0.497157,-0.578020,True,0,1,0,1,0,1,0,0,1,0,0,0,1
3,2.296006,-0.530599,-0.497157,-0.554231,True,0,1,0,1,0,0,0,1,1,0,0,0,1
4,0.313527,-0.530599,-0.497157,-0.550386,True,0,1,0,1,0,0,0,1,1,0,1,0,0
